In [1]:
# 웹 관련 임포트
import time
import requests
import urllib
from urllib.request import urlopen
from html_table_parser import parser_functions

from bs4 import BeautifulSoup
from selenium import webdriver # version_3.14.1
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.keys import Keys

# 데이터 프레임 관련 임포트
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

# 경고 및 한글처리
import warnings
warnings.filterwarnings('ignore')
plt.rc('font', family='Malgun Gothic')

### 최종

In [ ]:
# 데이터 프레임 관련 임포트
import numpy as np
import pandas as pd
from tqdm import tqdm

# 웹 관련 임포트
import requests
import urllib
from selenium import webdriver # version_3.14.1
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.keys import Keys

# 한글처리
import matplotlib.pyplot as plt
plt.rc('font', family='Malgun Gothic')

# 0. 페이지 오픈
driver = webdriver.Chrome('./driver/chromedriver.exe')
driver.get('https://www.elis.go.kr/allalr/allAlrList')

# 1. 선택 정보 추출
# 시도 정보 추출
def getSido():
    global sido_names
    
    sido_list_selectbox = driver.find_element_by_xpath('//*[@id="ctpvCd"]')
    sido_list = sido_list_selectbox.find_elements_by_tag_name('option')
    sido_names = [sido.text for sido in sido_list]

# id값 으로 버튼 클릭
def clickChecktBox():
    num = 0
    while True:
        try:
            # 조 목록 가져오기
            num += 1
            jo = driver.find_element_by_xpath(f'/html/body/div/div/div/div[2]/div[2]/div[2]/div/span[{num}]').text

            # 조 목록에 용적률, 건폐율이 포함된 조항만 체크박스 선택
            if '용적률' in jo or '건폐율' in jo:
                joid = driver.find_element_by_xpath(f'/html/body/div/div/div/div[2]/div[2]/div[2]/div/span[{num}]/label').get_attribute('for')
                button = driver.find_element_by_id(joid)
                driver.execute_script('arguments[0].click();', button)
            else:
                pass
        except:
            break
            pass        

# 2. 도시계획 조례 txt 다운로드
def getTxt(sido):
    # 시도 선택
    element = driver.find_element_by_id('ctpvCd')
    element.send_keys(sido)
    time.sleep(2)
    
    # 시군구 > 본청 선택
    element = driver.find_element_by_id('sggCd')
    element.send_keys('본청')

    # 법규구분 > 조례 선택
    driver.find_element_by_xpath('//*[@id="srchForm"]/div[1]/div[1]/div/div/table/tbody/tr[1]/td[2]/span[2]').click()

    # 검색어 입력
    searchByName = driver.find_element_by_xpath('//*[@id="edtSrchKwd"]')
    searchByName.clear()
    searchByName.send_keys('도시계획 조례')
    search_click = driver.find_element_by_xpath('//*[@id="btnSrch"]').click()

    # 도시계획 조례 클릭
    driver.find_element_by_xpath('//*[@id="srchForm"]/div[2]/div/div[4]/table/tbody/tr[1]/td[2]/a').click()

    # txt 저장 버튼 클릭
    driver.find_element_by_xpath('//*[@id="btnSave"]').click()
    time.sleep(2)
    driver.switch_to_window(driver.window_handles[1]) # 새창 전환
    driver.find_element_by_xpath('/html/body/div/div/div/div[2]/div[1]/div/span/label').click()
    driver.find_element_by_xpath('/html/body/div/div/div/div[2]/div[2]/div[1]/span/label').click()
    driver.find_element_by_xpath('/html/body/div/div/div/div[2]/div[3]/div/span/label').click()

    # 목록 이름에 용적률, 건폐율이 포함된 조항만 선택
    clickChecktBox()

    # txt 저장
    driver.find_element_by_xpath('/html/body/div/div/div/div[1]/a[1]').click()
    time.sleep(5)

    # 창 닫기, 검색 창으로 전환
    driver.close()
    driver.switch_to_window(driver.window_handles[0])
    time.sleep(1)
    driver.back()

# 3. 전 지역 도시계획 조례 txt 다운로드 실행
getSido()

for sido in tqdm(sido_names):
    try:
        getTxt(sido)
    except:
        print(sido)
        pass

driver.close()

### 자치법규정보시스템 크롤링

In [163]:
# 0. 페이지 오픈
driver = webdriver.Chrome('./driver/chromedriver.exe')
driver.get('https://www.elis.go.kr/allalr/allAlrList')

In [164]:
# 1. 선택 정보 추출
# 시도 정보 추출
def getSido():
    global sido_names
    
    sido_list_selectbox = driver.find_element_by_xpath('//*[@id="ctpvCd"]') # 셀렉트 박스 선택
    sido_list = sido_list_selectbox.find_elements_by_tag_name('option') # 셀렉트 박스 안에 있는 옵션 선택
    sido_names = [sido.text for sido in sido_list] # 시도명 리스트에 담기

# id값 으로 버튼 클릭
def clickChecktBox():
    num = 0
    while True:
        try:
            # 조 목록 가져오기
            num += 1
            jo = driver.find_element_by_xpath(f'/html/body/div/div/div/div[2]/div[2]/div[2]/div/span[{num}]').text

            # 조 목록에 용적률, 건폐율이 포함된 조항만 체크박스 선택
            if '용적률' in jo or '건폐율' in jo:
                joid = driver.find_element_by_xpath(f'/html/body/div/div/div/div[2]/div[2]/div[2]/div/span[{num}]/label').get_attribute('for') # 해당 조항의 id값 가져오기
                button = driver.find_element_by_id(joid) # id값 선택
                driver.execute_script('arguments[0].click();', button) # 체크박스 클릭
            else:
                pass
        except:
            break
            pass

In [184]:
# 2. 도시계획 조례 txt 다운로드
def getTxt(sido):
    # 시도 선택
    element = driver.find_element_by_id('ctpvCd')
    element.send_keys(sido)
    
    time.sleep(2)
    
    # 시군구 > 본청 선택
    element = driver.find_element_by_id('sggCd')
    element.send_keys('본청')

    # 법규구분 > 조례 선택
    driver.find_element_by_xpath('//*[@id="srchForm"]/div[1]/div[1]/div/div/table/tbody/tr[1]/td[2]/span[2]').click()

    # 검색어 입력
    searchByName = driver.find_element_by_xpath('//*[@id="edtSrchKwd"]')
    searchByName.clear()
    searchByName.send_keys('도시계획 조례')
    search_click = driver.find_element_by_xpath('//*[@id="btnSrch"]').click()

    # 도시계획 조례 클릭
    driver.find_element_by_xpath('//*[@id="srchForm"]/div[2]/div/div[4]/table/tbody/tr[1]/td[2]/a').click()

    # txt 저장 버튼 클릭
    driver.find_element_by_xpath('//*[@id="btnSave"]').click()
    
    time.sleep(2)
    
    driver.switch_to_window(driver.window_handles[1]) # 새창 전환
    driver.find_element_by_xpath('/html/body/div/div/div/div[2]/div[1]/div/span/label').click() # 전체선택 해제
    driver.find_element_by_xpath('/html/body/div/div/div/div[2]/div[2]/div[1]/span/label').click() # 전체선택 해제
    driver.find_element_by_xpath('/html/body/div/div/div/div[2]/div[3]/div/span/label').click() # 부칙선택 해제

    # 목록 이름에 용적률, 건폐율이 포함된 조항만 선택
    clickChecktBox()

    # txt 저장
    driver.find_element_by_xpath('/html/body/div/div/div/div[1]/a[1]').click()
    
    time.sleep(5)

    # 창 닫기, 검색 창으로 전환
    driver.close()
    driver.switch_to_window(driver.window_handles[0])
    
    time.sleep(1)
    
    driver.back() # 뒤로가기

In [185]:
# 3. 전 지역 도시계획 조례 txt 다운로드 실행
getSido()

for sido in tqdm(sido_names):
    try:
        getTxt(sido)
    except:
        print(sido)
        pass

driver.close()

 95%|█████████████████████████████████████████████████████████████████████████████▋    | 18/19 [03:47<00:12, 12.62s/it]


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="srchForm"]/div[2]/div/div[4]/table/tbody/tr[1]/td[2]/a"}
  (Session info: chrome=104.0.5112.102)


###### 참고

In [ ]:
# 텍스트로 가져오기
driver = webdriver.Chrome('./driver/chromedriver.exe')
driver.get('https://cu.bgfretail.com/store/list.do?category=store')

corp_list = [] # 매장명
addr_list = [] # 주소

# '점'으로 검색
searchByName = driver.find_element_by_xpath('//*[@id="jumpoName"]')
searchByName.clear()
searchByName.send_keys('점')
click = driver.find_element_by_xpath('//*[@id="contents"]/div[1]/div/div[1]/div[2]/table/tbody/tr[2]/td/a').click()

time.sleep(1)

for i in tqdm(range(1000)):
    for corp_index in range(1,6):
        corp = driver.find_element_by_xpath(f'//*[@id="result_search"]/div[2]/div[1]/table/tbody/tr[{corp_index}]/td[1]/span[1]').text
        addr = driver.find_element_by_xpath(f'//*[@id="result_search"]/div[2]/div[1]/table/tbody/tr[{corp_index}]/td[2]/div/address/a').text
        
        time.sleep(1)
        
        corp_list.append(corp)
        addr_list.append(addr)
        
    # 다음 목록 이동
    paging = driver.find_element_by_xpath('//*[@id="paging"]/a').get_attribute('text')
    if paging == '다음':
        nextPage = driver.find_element_by_xpath(f'//*[@id="paging"]/a').click()
    elif driver.find_element_by_xpath(f'//*[@id="paging"]/a[2]').get_attribute('text') == '다음':
        nextPage = driver.find_element_by_xpath(f'//*[@id="paging"]/a[2]').click()
    else :
        pass

    time.sleep(1)

driver.close()

In [ ]:
df = pd.DataFrame({'corp_list':corp_list,'addr_list':addr_list})

In [ ]:
df

### 파일 한 번에 불러오기

In [2]:
from glob import glob

In [121]:
glob('./*도시계획 조례.txt')

['.\\강원도 도시계획 조례.txt',
 '.\\경기도 도시계획 조례.txt',
 '.\\경상남도 도시계획 조례.txt',
 '.\\경상북도 도시계획 조례.txt',
 '.\\광주광역시 도시계획 조례.txt',
 '.\\대구광역시 도시계획 조례.txt',
 '.\\대전광역시 도시계획 조례.txt',
 '.\\부산광역시 도시계획 조례.txt',
 '.\\서울특별시 도시계획 조례.txt',
 '.\\세종특별자치시 도시계획 조례.txt',
 '.\\울산광역시 도시계획 조례.txt',
 '.\\인천광역시 도시계획 조례.txt',
 '.\\전라남도 도시계획 조례.txt',
 '.\\전라북도 도시계획 조례.txt',
 '.\\제주특별자치도 도시계획 조례.txt',
 '.\\충청남도 도시계획 조례.txt',
 '.\\충청북도 도시계획 조례.txt']

In [197]:
# txt to dataframe
tmp_raw = []
station_raw = pd.DataFrame()

for file_name in glob('./*도시계획 조례.txt'):
    with open(file_name, encoding='utf-8') as f:
        lines = f.readlines()
        df = pd.DataFrame(lines, columns=[file_name.split(' ')[0][2:]]).T
        
        # 용적률, 건폐율 데이터 프레임 나눠담기
        
        # 퍼센트 남기기
        
        
    station_raw = pd.concat([station_raw,df])


In [198]:
station_raw.T

,강원도,경기도,경상남도,경상북도,광주광역시,대구광역시,대전광역시,부산광역시,서울특별시,세종특별자치시,울산광역시,인천광역시,전라남도,전라북도,제주특별자치도,충청남도,충청북도
0,제6조의3(도시지역 내 지구단위계획구역에서의 건폐율 등의 완화적용) ① 삭제<202...,제6조의6(도시지역 내 지구단위계획구역에서의 용적률 완화) 영 제46조제11항에서 ...,제6조의6(도시지역 내 지구단위계획구역에서의 건폐율 등의 완화적용 <개정 2020...,제7조의2(도시지역 내 지구단위계획구역에서의 건폐율 등의 완화적용) ① 영 제46조...,제25조의2(도시지역 내 지구단위계획구역에서의 건폐율 등의 완화적용) ① 삭제 <2...,제17조의2(지구단위계획구역 안에서의 건폐율 등 완화)(개정 2012. 11. 12...,제19조(지구단위계획구역 안에서의 건폐율 등의 완화적용) ① 삭제 <2022.2.1...,제36조(경관지구안에서의 건폐율 등) ① 영 제72조제2항에 따라 자연경관지구안에서...,\n,"제19조(지구단위계획구역안에서의 건폐율 등의 완화적용) ①<삭제, 2020. 12....",제16조의3(지구단위계획구역에서의 건폐율 등의 완화적용) ① 영 제46조제1항에 따...,제53조(경관지구의 건폐율 및 건축선 후퇴 부분의 제한 등) ① 영 제72조제2항에...,NaN,제6조의3(지구단위계획구역에서의 건폐율 등의 완화적용) ① 영 제46조제1항의 규정...,\n,제6조(도시지역 내 지구단위계획구역에서의 건폐율 등의 완화적용) ① 삭제<2021....,제7조의2(도시지역 내 지구단위계획구역에서의 건폐율 등의 완화 적용) ① 삭제<20...
1,② 영 제46조제1항제2호에 따른 공공시설등의 설치비용과 부지가액의 산정방법은 ...,[본조신설 2021.11.2.]\n,② 영 제46조제1항제2호 후단에서 도시·군계획조례로 정하는 공공시설등의 설치비...,1. 공공시설등의 설치비용은 원가계산용역기관에서 해당 시설 설치에 소요되는 재료...,② 영 제46조제1항제2호 공공시설등(법 제52조의2제1항의 공공시설 및 기반시...,1. 「공공주택특별법」 제2조제1호가목에 따른 공공임대주택\n,② 지구단위계획구역 안에서 건축물을 건축하려는 자가 그 대지의 일부에 설치하는 ...,1. 건폐율 : 40퍼센트 이하\n,제1장 총칙\n,② 영 제46조제1항제2호 후단에 따라 조례로 정하는 공공시설등 설치비용 및 이...,② 삭제 <2019. 11. 7.>\n,② 수변특화경관지구안에서의 건축하는 건축물의 건폐율은 50퍼센트를 초과할 수 없...,NaN,② 영 제46조제1항제2호에 따라 도시계획 조례로 정하도록 한 공공시설등의 설치...,제3절 도시계획시설\n,② 영 제46조제1항제2호에 따른 공공시설 등의 설치비용과 부지 가액의 산정방법...,② 영 제46조제1항제2호에 따른 공공시설등의 설치비용과 부지 가액의 산정 방법...
2,1. 공공시설등의 설치비용은 「지방자치단체를 당사자로 하는 계약에 관한 법률 시...,\n,1. 공공시설등의 설치비용은「지방자치단체를 당사자로 하는 계약에 관한 법률 시행...,2. 부지가액은 「감정평가 및 감정평가사에 관한 법률」에 따른 감정평가업자 2명...,"1. 공공시설등 설치비용은 시설설치에 드는 재료비, 노무비, 경비 등을 고려하여...",2. 「건축법 시행령」 별표1제2호라목에 따른 기숙사\n,③ 영 제46조제1항제2호에 따라 “조례로 정하는 공공시설등 설치비용 및 이에 ...,2. 건축물의 높이 : 15미터 이하\n,\n,"1. 공공시설등 설치비용은 시설설치에 소요되는 노무비, 재료비, 경비 등을 고려...",③ 법 제52조의2제1항제3호에서 “도시계획조례로 정하는 시설”이란 공공 필요성...,③ 경관지구안에 건축하는 건축물의 높이는 경관지구의 세분에 따라 다음 각 호에 ...,NaN,1. 공공시설등의 설치비용은 「지방자치단체를 당사자로 하는 계약에 관한 법률 시...,제16조(도시지역 내 지구단위계획구역에서의 건폐율 등의 완화적용) ① 제주특별법 제...,"1. 공공시설등의 설치비용은 해당 시설의 설치에 소요되는 재료비, 노무비, 경비...","1. 공공시설등의 설치비용은 해당 시설의 설치에 드는 재료비, 노무비, 경비 등..."
3,2. 부지가액은 국토교통부장관이 정하는 요건을 갖춘 「감정평가 및 감정평가사에 ...,NaN,2. 부지 가액은 「감정평가 및 감정평가사에 관한 법률」 제2조제4호에 따른 ...,3. 공공시설등의 설치비용 및 부지가액의 산정은 건축허가 신청일을 기준으로 한다...,2. 부지가액은 둘 이상의 감정평가법인등이 평가한 감정평가가액의 평균가를 원칙으...,② 영 제46조제1항제2호에 따라 공공시설등을 설치하여 제공(그 부지의 제공은 ...,"1. 공공시설등 설치비용은 시설설치에 소요되는 노무비, 재료비, 경비 등을 고려...",3. 대지안의 조경 : 대지면적의 30퍼센트 이상\n,제2장 광역도시계획 및 도시기본계획\n,2. 부지 가액은 「감정평가 및 감정평가사에 관한 법률」에 따른 감정평가법인등(...,1. 「공공주택특별법」 제2조제1호가목에 따른 공공임대주택\n,1. 자연경관지구안에 건축하는 건축물의 높이는 4층 이하로써 14미터를 초과하여...,NaN,2. 부지 가액은 국토교통부 장관이 정하는 요건을 갖춘 「부동산 가격공시 및 감...,1. 공공시설등의 부지를 제공하는 경우에는 다음 각 목의 비율까지 건폐율·용적률...,2. 부지 가액은 「감정평가 및 감정평가사에 관한 법률」 제2조제4호에 따른 감...,2. 부지 가액은 「부동산 가격공시에 관한 법률」에 따른 감정평가업자 2명 이상...
4,3. 공공시설 등의 설치비용 및 부지가액의 산정은 건축허가 신청일을 기준으로 한...,NaN,3. 제1호의 설치비용 및 제2호의 부지 가액 산정에 소요되는 비용은 공공시설등...,4. 공공시설등의 설치이행을 담보하기 위하여 이행보증금을 예치하게 할 수 있으며...,③ 제2항의 산정방법 등에 필요한 사항은 규칙으로 따로 정한다.<개정 2013....,"1. 공공시설등의 설치비용은 해당 시설의 설치에 소요되는 노무비, 재료비, 경비...",2. 부지 가액은 개별공시지가와 인근 지역의 실거래가를 기준으로 산정한다. 다만...,② 영 제72조제2항에 따라 특화경관지구안에 건축하는 건축물의 높이는 12미터를...,\n,③ 제2항의 산정방법 등 시행에 필요한 사항은 규칙으로 정한다.\n,2. 「건축법 시행령」 별표 1 제2호라목에 따른 기숙사\n,2. 수변특화경관지구안에 건축하는 건축물의 높이는 5층 이하로써 17미터를 초과...,NaN,3. 제1호 및 제2호에도 불구하고 구체적인 개발안을 전제로 지구단위계획을 수립...,가. 완화할 수 있는 건폐율= 해당 용도지역에 적용되는 건폐율 × [1 + ...,3. 공공시설등의 설치비용과 부지 가액의 산정은 사업시행자가 건축허가를 신청한 ...,3. 공공시설등의 설치비용 및 부지 가액의 산정은 건축허가 시점을 기준으로 한다...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,제10장 보칙 <개정 2011. 7. 28.>\n,NaN,NaN,NaN,NaN,NaN,"2. 「도시 및 주거환경정비법」에 따른 주택재개발사업, 주택재건축사업 및 도시환...",NaN,NaN
150,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,\n,NaN,NaN,NaN,NaN,NaN,"⑦ 제주특별법 제406조제9항에 따라 법 제78조제5항에서 ""대통령령으로 정하는...",NaN,NaN
151,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,⑧ 법 제78조제6항에 따라 완화할 수 있는 용적률은 제1항 용적률의 120퍼센...,NaN,NaN
152,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,⑨ 영 제85조제10항제3호에 따른 “도시·군계획 조례로 정하는 사회복지지설”이...,NaN,NaN


In [161]:
with open('.\\서울특별시 도시계획 조례.txt', encoding='utf-8') as f:
    lines = f.readlines()

In [201]:
df = pd.DataFrame(lines, columns=[file_name]).drop_duplicates(keep=False).reset_index(drop=True)
df[file_name] = df[file_name].str.replace('\n','')
df.T

,0,1,2,3,4,5,6,7,8,9,...,121,122,123,124,125,126,127,128,129,130
.\충청북도 도시계획 조례.txt,제1장 총칙,제2장 광역도시계획 및 도시기본계획,제3장 도시관리계획의 입안,제4장 용도지구의 지정,제5장 도시계획시설의 관리,제6장 지구단위계획,제7장 개발행위의 허가,제8장 용도지역ㆍ용지지구 및 용도구역안에서의 행위제한 <개정 2011. 7. 28.>,제1절 용도지역안에서의 행위제한,제2절 경관지구안에서의 건축제한,...,2. 도시계획시설인 학교 부지 내의 기숙사,㉔ 제1항에도 불구하고 감염병 대응 등을 위하여 필요한 경우에는 다음 각 호의 ...,1. 영 제85조제3항제6호에 의한 경우에는 영 제85조제1항 각 호에 따른 용...,2. 도시계획시설인 종합의료시설 부지(도시계획시설 대학 내 의료시설 포함)에 지...,제6절 기존 건축물의 특례 <신설 2018. 10. 4.>,제7절 개발제한구역의 관리 <신설 2018. 10. 4.>,제9장 서울특별시 도시계획위원회,제1절 서울특별시 도시계획위원회의 운영 등,제2절 도시계획상임기획단,제10장 보칙 <개정 2011. 7. 28.>


In [196]:
df

,0,1,2,3,4,5,6,7,8,9
충청북도,제7조의2(도시지역 내 지구단위계획구역에서의 건폐율 등의 완화 적용) ① 삭제<20...,② 영 제46조제1항제2호에 따른 공공시설등의 설치비용과 부지 가액의 산정 방법...,"1. 공공시설등의 설치비용은 해당 시설의 설치에 드는 재료비, 노무비, 경비 등...",2. 부지 가액은 「부동산 가격공시에 관한 법률」에 따른 감정평가업자 2명 이상...,3. 공공시설등의 설치비용 및 부지 가액의 산정은 건축허가 시점을 기준으로 한다...,4. 제1호 및 제2호에 드는 비용은 공공시설등을 설치·제공하는 자가 부담한다.\n,5. 그 밖에 시행에 필요한 운영기준 등은 시장·군수가 정할 수 있다.\n,[본조신설 2017. 7. 7]\n,③ 영 제46조제11항에 따른 “도시ㆍ군계획조례로 정하는 비율”은 10퍼센트로 ...,\n


In [183]:
df[df[file_name].str.contains('퍼센트')]

,.\충청북도 도시계획 조례.txt
14,1. 제1종전용주거지역：50퍼센트
15,2. 제2종전용주거지역：40퍼센트
16,3. 제1종일반주거지역：60퍼센트
17,4. 제2종일반주거지역：60퍼센트
18,5. 제3종일반주거지역：50퍼센트
...,...
110,1. 제1항에 따른 용적률의 120퍼센트
113,㉒ 제1항제6호 규정에도 불구하고 준주거지역에서는 「공공주택 특별법」 제2조제1...
114,㉓ 제1항제2호부터 제6호까지에 따른 지역에서 「고등교육법」 제2조에 따른 학교...
123,1. 영 제85조제3항제6호에 의한 경우에는 영 제85조제1항 각 호에 따른 용...


In [182]:
if '퍼센트' in df[file_name]:
    

0                                제1장 총칙
1                   제2장 광역도시계획 및 도시기본계획
2                        제3장 도시관리계획의 입안
3                          제4장 용도지구의 지정
4                        제5장 도시계획시설의 관리
                     ...               
126    제7절 개발제한구역의 관리 <신설 2018. 10. 4.>
127                   제9장 서울특별시 도시계획위원회
128             제1절 서울특별시 도시계획위원회의 운영 등
129                       제2절 도시계획상임기획단
130           제10장 보칙 <개정 2011. 7. 28.>
Name: .\충청북도 도시계획 조례.txt, Length: 131, dtype: object

In [173]:
df.columns = ['충북']

In [ ]:
station_raw = pd.concat(tmp_raw)

### txt to Dataframe

###### 방법1 (text read)

In [6]:
# read txt file
f = open('./서울특별시 도시계획 조례.txt', 'rt', encoding='utf-8')
data = f.read()

In [19]:
data

'\n제1장 총칙\n\n제2장 광역도시계획 및 도시기본계획\n\n제3장 도시관리계획의 입안\n\n제4장 용도지구의 지정\n\n제5장 도시계획시설의 관리\n\n제6장 지구단위계획\n\n제7장 개발행위의 허가\n\n제8장 용도지역ㆍ용지지구 및 용도구역안에서의 행위제한 <개정 2011. 7. 28.>\n\n제1절 용도지역안에서의 행위제한\n\n제2절 경관지구안에서의 건축제한\n\n제3절 보호지구안에서의 건축제한 <신설 2018. 10. 4.>\n\n제4절 그 밖의 용도지구안에서의 건축제한 <신설 2018. 10. 4.>\n\n제5절 건폐율 및 용적률 <신설 2018. 10. 4.>\n제54조(용도지역안에서의 건폐율) ① 법 제77조 및 영 제84조제1항에 따라 용도지역별 건폐율은 다음 각 호의 비율 이하로 한다. <개정 2008. 7. 30.>\n  1. 제1종전용주거지역：50퍼센트\n  2. 제2종전용주거지역：40퍼센트\n  3. 제1종일반주거지역：60퍼센트\n  4. 제2종일반주거지역：60퍼센트\n  5. 제3종일반주거지역：50퍼센트\n  6. 준주거지역：60퍼센트\n  7. 중심상업지역：60퍼센트\n  8. 일반상업지역：60퍼센트\n  9. 근린상업지역：60퍼센트\n  10. 유통상업지역：60퍼센트\n  11. 전용공업지역：60퍼센트\n  12. 일반공업지역：60퍼센트\n  13. 준공업지역：60퍼센트\n  14. 보전녹지지역：20퍼센트\n  15. 생산녹지지역：20퍼센트\n  16. 자연녹지지역：20퍼센트\n  ② 법 제77조제3항 및 영 제84조제4항에 따라 다음 각 호의 지역 안에서의 건폐율은 제1항에도 불구하고 다음 각 호의 비율 이하로 한다. <개정 2008. 7. 30., 2011. 7. 28., 2012. 11. 1., 2016. 9. 29., 2019. 7. 18., 2020. 7. 16.>\n  1. 취락지구 : 60퍼센트(집단취락지구에 대해서는 「개발제한구역의 지정 및 관리에 관한 특별조치법령」 이 정하는 바에 따른다)\n  2. 「자연공원법

###### 방법2 (txt to dataframe)

In [35]:
# txt to dataframe
df = pd.read_csv('./서울특별시 도시계획 조례.txt', sep='\t', header=None)

In [36]:
df.head(20)

,0
0,제1장 총칙
1,제2장 광역도시계획 및 도시기본계획
2,제3장 도시관리계획의 입안
3,제4장 용도지구의 지정
4,제5장 도시계획시설의 관리
5,제6장 지구단위계획
6,제7장 개발행위의 허가
7,제8장 용도지역ㆍ용지지구 및 용도구역안에서의 행위제한 <개정 2011. 7. 28.>
8,제1절 용도지역안에서의 행위제한
9,제2절 경관지구안에서의 건축제한


In [336]:
# 55조(용적률) 찾기
df[df[0].str.contains('퍼센트')].head(20)

,0
14,1. 제1종전용주거지역：50퍼센트
15,2. 제2종전용주거지역：40퍼센트
16,3. 제1종일반주거지역：60퍼센트
17,4. 제2종일반주거지역：60퍼센트
18,5. 제3종일반주거지역：50퍼센트
19,6. 준주거지역：60퍼센트
20,7. 중심상업지역：60퍼센트
21,8. 일반상업지역：60퍼센트
22,9. 근린상업지역：60퍼센트
23,10. 유통상업지역：60퍼센트


###### 참고

In [45]:
# !pip install python-docx

     ---------------------------------------- 5.6/5.6 MB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for python-docx: filename=python_docx-0.8.11-py3-none-any.whl size=184507 sha256=024d0d997fff1fca25290e949cf49b65e4d90431a5d772d6f58dd216a98494f4
  Stored in directory: c:\users\dupar\appdata\local\pip\cache\wheels\32\b8\b2\c4c2b95765e615fe139b0b17b5ea7c0e1b6519b0a9ec8fb34d
Successfully built python-docx


In [50]:
# import docx NOT python-docx
from docx import Document
# create an instance of a word document
doc = docx.Document()

In [55]:
# doc 파일 불러오기
# doc = Document('./서울특별시 도시계획 조례(서울특별시조례)(제8435호)(20220711).doc')
doc = Document(r'C:\oasis\서울특별시 도시계획 조례(서울특별시조례)(제8435호)(20220711).doc')

PackageNotFoundError: Package not found at 'C:\oasis\서울특별시 도시계획 조례(서울특별시조례)(제8435호)(20220711).doc'